In [2]:
import cv2
import numpy as np
import os
from random import shuffle
from tqdm import tqdm

In [3]:
TRAIN_DIR = './datasets/train'
TEST_DIR = './datasets/test'
IMG_SIZE = 50
LR = 0.001

In [4]:
MODEL_NAME = 'dogsvscats-{}-{}.model'.format(LR, '2conv-basic')

In [5]:
def label_img(img):
    word_label = img.split('.')[-3]
    if word_label == 'cat': return [1, 0]
    elif word_label == 'dog': return [0, 1]

In [6]:
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_dir, img)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        
        training_data.append([np.array(img), np.array(label)])
        
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data